<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2020/blob/master/02_lab/kaggle_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up variables and download data

Register on [kaggle](https://www.kaggle.com) and accept the [competition](https://www.kaggle.com/t/f62a416111034502a0090ea0c6adcf18) rules.

Go to My Account and under API section click **create new API Token**.
Download created kaggle.json

Upload this file to your google drive root folder.

Now execute the following magic. - It installs kaggle, mounts google drive and downloads data from competition to you drive.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir /root/.kaggle
!cp /content/gdrive/My\ Drive/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!ls -l /root/.kaggle

total 4
-rw------- 1 root root 64 Jan 28 12:02 kaggle.json


In [0]:
DATA_PATH = "/content/gdrive/My Drive/mlimperial2020-1"

In [0]:
ls /content/gdrive/My\ Drive/mlimperial2020-1

macro.csv.zip  X_test.csv.zip  X_train.csv.zip


In [0]:
#!kaggle config set -n path -v /content
!kaggle competitions download -c mlimperial2020-1 -p '/content/gdrive/My Drive/mlimperial2020-1'

X_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
X_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
macro.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


# https://www.kaggle.com/t/f62a416111034502a0090ea0c6adcf18

### Metric

For regression task we can use the most common Mean Squared Error(MSE). However, sometimes its better to use logarithmic error. In this challenge, we will use RMSLE - root mean square logarithmic error:

$$
RMSLE = \sqrt{\frac{1}{N} \sum_{i=1}^{N} [\log(y_i + 1) - \log(p_i + 1)]^2},
$$

where $y_i$ is true value and $p_i$ is a predicted value.

# Grading

*TBA*

# Baseline

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error,mean_squared_error
import os

In [0]:
data = pd.read_csv(os.path.join(DATA_PATH, 'X_train.csv.zip'), parse_dates=['timestamp'])
test = pd.read_csv(os.path.join(DATA_PATH, 'X_test.csv.zip'), parse_dates=['timestamp'])
macro = pd.read_csv(os.path.join(DATA_PATH, "macro.csv.zip"), parse_dates=['timestamp'])

In [0]:
data.shape, test.shape, macro.shape

((21329, 292), (9142, 291), (2484, 100))

In [0]:
data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,11180,2013-10-11,51,30.0,2.0,17.0,1.0,1986.0,2.0,8.0,3.0,Investment,Chertanovo Severnoe,6.206099e+06,111874,0.128123,0.089040,4720,2881.0,5,7976,6784.0,5,0,NaN,0,0,10,3,no,0,11,5,no,no,no,no,no,no,no,...,675.32,1142.86,909.09,6,25,26,17,8,1,0,2,10,1,2,24,8,23.89,19.10,38,1816638,41,1525045,156,668.75,1125.00,896.88,12,46,47,37,13,1,0,7,33,1,9,48,11,9300000
1,23071,2014-09-30,48,48.0,9.0,17.0,4.0,2016.0,1.0,10.0,1.0,OwnerOccupier,Izmajlovo,1.504557e+07,102828,0.637399,0.006076,4992,1313.0,4,5285,4339.0,6,0,645.0,6,0,16,3,no,0,6,0,no,no,no,yes,no,no,no,...,675.86,1137.93,906.90,4,31,26,19,10,1,0,7,8,0,4,41,2,21.83,16.82,110,1858453,47,1043785,291,691.70,1175.09,933.39,14,77,105,63,27,5,0,25,44,0,12,114,13,5277083
2,18942,2014-05-27,77,46.0,2.0,16.0,1.0,1978.0,3.0,10.0,NaN,Investment,Jasenevo,2.481385e+07,174831,0.683844,0.037178,10712,4172.0,7,11217,10559.0,7,0,2300.0,1,0,7,7,no,0,7,1,no,no,no,yes,no,no,no,...,515.00,875.00,695.00,1,11,5,3,1,0,0,2,7,0,0,9,4,45.72,6.45,9,178712,49,1563795,89,627.38,1059.52,843.45,5,29,28,21,6,0,0,5,22,0,1,33,8,12500000
3,24805,2014-11-17,109,54.0,9.0,14.0,1.0,2001.0,3.0,11.0,2.0,Investment,Juzhnoe Butovo,2.615514e+07,178264,0.137846,0.041116,14080,11926.0,11,14892,24750.0,13,1,NaN,1,0,13,4,no,0,4,4,no,no,no,no,no,no,no,...,776.92,1230.77,1003.85,1,2,3,8,0,0,0,1,3,1,0,9,0,26.59,5.31,0,0,10,308554,20,788.89,1277.78,1033.33,2,4,5,8,0,1,0,1,10,1,0,16,0,13050000
4,14778,2014-02-10,33,19.0,2.0,3.0,2.0,1961.0,1.0,6.0,3.0,Investment,Sviblovo,4.441296e+06,60315,0.137976,0.178917,3694,2057.0,1,3594,3741.0,2,0,192.0,1,0,4,6,no,0,5,4,no,no,no,no,no,no,no,...,670.00,1131.25,900.63,6,27,26,18,8,1,0,6,11,1,2,37,2,24.65,14.40,48,836992,34,1308036,229,683.33,1159.52,921.43,19,70,70,41,26,3,0,14,29,1,4,90,6,5500000


In [0]:
macro.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,eurrub,brent,net_capital_export,gdp_annual,gdp_annual_growth,average_provision_of_build_contract,average_provision_of_build_contract_moscow,rts,micex,micex_rgbi_tr,micex_cbi_tr,deposits_value,deposits_growth,deposits_rate,mortgage_value,mortgage_growth,mortgage_rate,grp,grp_growth,income_per_cap,real_dispos_income_per_cap_growth,salary,salary_growth,fixed_basket,retail_trade_turnover,retail_trade_turnover_per_cap,retail_trade_turnover_growth,labor_force,unemployment,employment,...,sewerage_share,gas_share,hot_water_share,electric_stove_share,heating_share,old_house_share,average_life_exp,infant_mortarity_per_1000_cap,perinatal_mort_per_1000_cap,incidence_population,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,child_on_acc_pre_school,load_of_teachers_school_per_teacher,students_state_oneshift,modern_education_share,old_education_build_share,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38807.2,-0.078209,5.0,NaN,NaN,NaN,NaN,NaN,7484970,NaN,NaN,142968,NaN,13.72,8375.8638,NaN,30789.2,0.038,38410.5,NaN,11443.63,2882.4169,251.484,106.8,6436.244,0.018,0.69,...,99.6,44.4,95.7,55.0,99.9,0.3,74.2,6.1,5.87,696.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721.477765,"45,713",1356.112607,NaN,NaN,NaN,18.0,99.4,7872.848285,162.9,NaN,NaN,NaN,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38807.2,-0.078209,5.0,NaN,NaN,NaN,NaN,175.15,7484970,NaN,NaN,142968,NaN,13.72,8375.8638,NaN,30789.2,0.038,38410.5,NaN,11443.63,2882.4169,251.484,106.8,6436.244,0.018,0.69,...,99.6,44.4,95.7,55.0,99.9,0.3,74.2,6.1,5.87,696.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721.477765,"45,713",1356.112607,NaN,NaN,NaN,18.0,99.4,7872.848285,162.9,NaN,NaN,NaN,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38807.2,-0.078209,5.0,NaN,NaN,NaN,NaN,178.66,7484970,NaN,NaN,142968,NaN,13.72,8375.8638,NaN,30789.2,0.038,38410.5,NaN,11443.63,2882.4169,251.484,106.8,6436.244,0.018,0.69,...,99.6,44.4,95.7,55.0,99.9,0.3,74.2,6.1,5.87,696.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721.477765,"45,713",1356.112607,NaN,NaN,NaN,18.0,99.4,7872.848285,162.9,NaN,NaN,NaN,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,43.4054,80.12,NaN,38807.2,-0.078209,5.0,NaN,NaN,NaN,NaN,183.44,7484970,NaN,NaN,142968,NaN,13.72,8375.8638,NaN,30789.2,0.038,38410.5,NaN,11443.63,2882.4169,251.484,106.8,6436.244,0.018,0.69,...,99.6,44.4,95.7,55.0,99.9,0.3,74.2,6.1,5.87,696.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721.477765,"45,713",1356.112607,NaN,NaN,NaN,18.0,99.4,7872.848285,162.9,NaN,NaN,NaN,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,42.9600,80.59,NaN,38807.2,-0.078209,5.0,NaN,NaN,NaN,NaN,183.44,7484970,NaN,NaN,142968,NaN,13.72,8375.8638,NaN,30789.2,0.038,38410.5,NaN,11443.63,2882.4169,251.484,106.8,6436.244,0.018,0.69,...,99.6,44.4,95.7,55.0,99.9,0.3,74.2,6.1,5.87,696.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721.477765,"45,713",1356.112607,NaN,NaN,NaN,18.0,99.4,7872.848285,162.9,NaN,NaN,NaN,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


As you can see, the timestamp is important here, because it will define the various variables, that change with time, for example, gdp or mortgage rate. Lets, for example, merge train, test,the data on the timestamp.

In [0]:
y_train = data["price_doc"]

data.drop(['id', 'price_doc'], axis=1, inplace=True)

# num_train = len(X_train)
# X_all = pd.concat([X_train, X_test])


X_all = pd.merge_ordered(data, macro, on='timestamp', how='left')


In [0]:
X_all.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,...,sewerage_share,gas_share,hot_water_share,electric_stove_share,heating_share,old_house_share,average_life_exp,infant_mortarity_per_1000_cap,perinatal_mort_per_1000_cap,incidence_population,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,child_on_acc_pre_school,load_of_teachers_school_per_teacher,students_state_oneshift,modern_education_share,old_education_build_share,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo,6.407578e+06,155572,0.189727,0.000070,9576,5001.0,5,10309,11065.0,5,0,240.0,1,0,7,3,no,0,16,1,no,no,no,no,no,no,no,no,...,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,#!,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
1,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,6880,3119.0,5,7759,6237.0,8,0,229.0,1,0,6,1,yes,1,3,0,no,no,no,no,no,no,no,no,...,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,#!,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
2,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,5879,1463.0,4,6207,5580.0,7,0,1183.0,1,0,5,1,no,0,0,1,no,no,no,yes,no,no,no,no,...,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,#!,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
3,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino,1.258354e+07,178473,0.194703,0.069753,13087,6839.0,9,13670,17063.0,10,0,NaN,1,0,17,6,no,0,11,4,no,no,no,no,no,no,no,no,...,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,155.22,94.02,66.28,51.15,44.25,37.73,30.63,793.319561,#!,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
4,2011-09-08,25,14.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Sokol'niki,1.032047e+07,57405,0.523439,0.042307,2459,933.0,5,2810,5050.0,5,0,4849.0,3,0,17,6,no,0,6,9,no,no,no,no,no,no,no,yes,...,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,155.22,94.02,66.28,51.15,44.25,37.73,30.63,793.319561,#!,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0


A small hint - do we really need all 389 columns? What is the distribution of the predicted data?

For now, lets split the training set to train/test and fit simpliest linear model on top of it. But before that we must get rid of NaNs!, beacause not algorithms can deal with them.

In [0]:
X_all.fillna(0, inplace=True)

In [0]:
training_ind, validation_ind = train_test_split(range(len(X_all)), random_state=11, train_size=0.10)

In [0]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_all.iloc[training_ind])

TypeError: ignored

# Error! Why?

Because we have there are exists categorial values in the table. For now, we will just drop them, but you should not! They might be important for the prediction result.

In [0]:
df_numeric = X_all.select_dtypes(exclude=['object'])
df_numeric.drop(["timestamp"], inplace=True, axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
df_numeric.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,...,lodging_sqm_per_cap,water_pipes_share,baths_share,sewerage_share,gas_share,hot_water_share,electric_stove_share,heating_share,old_house_share,average_life_exp,infant_mortarity_per_1000_cap,perinatal_mort_per_1000_cap,incidence_population,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,load_of_teachers_school_per_teacher,students_state_oneshift,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,6.407578e+06,155572,0.189727,0.000070,9576,5001.0,5,10309,11065.0,5,0,240.0,1,0,7,3,0,16,1,86206,40477,45729,21154,11007,10147,98207,52277,45930,36211,10580,25631,...,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,1391.710938,89.0495,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
1,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,9.589337e+06,115352,0.372602,0.049637,6880,3119.0,5,7759,6237.0,8,0,229.0,1,0,6,1,1,3,0,76284,34200,42084,15727,7925,7802,70194,35622,34572,29431,9266,20165,...,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,1391.710938,89.0495,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
2,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.808270e+06,101708,0.112560,0.118537,5879,1463.0,4,6207,5580.0,7,0,1183.0,1,0,5,1,0,0,1,101982,46076,55906,13028,6835,6193,63388,31813,31575,25292,7609,17683,...,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,136.11,77.93,62.89,47.85,41.80,36.77,29.07,793.319561,1391.710938,89.0495,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
3,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.258354e+07,178473,0.194703,0.069753,13087,6839.0,9,13670,17063.0,10,0,0.0,1,0,17,6,0,11,4,21155,9828,11327,28563,14680,13883,120381,60040,60341,29529,9083,20446,...,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,155.22,94.02,66.28,51.15,44.25,37.73,30.63,793.319561,1391.710938,89.0495,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
4,25,14.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1.032047e+07,57405,0.523439,0.042307,2459,933.0,5,2810,5050.0,5,0,4849.0,3,0,17,6,0,6,9,85956,40724,45232,5682,2925,2757,35003,17490,17513,16720,5351,11369,...,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,155.22,94.02,66.28,51.15,44.25,37.73,30.63,793.319561,1391.710938,89.0495,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(df_numeric.iloc[training_ind])

In [0]:
predictor = Ridge()
predictor.fit(X_train, np.log1p(y_train[training_ind]))

X_test = scaler.transform(df_numeric.iloc[validation_ind])
mean_squared_error(predictor.predict(X_test), np.log1p(y_train[validation_ind]), squared=False)

0.6451176136016002

In [0]:
%%time

param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}

gscv = GridSearchCV(predictor, param_grid, scoring='neg_root_mean_squared_error', cv=3, n_jobs=-1, verbose=1)
gscv.fit(X_train, np.log1p(y_train[training_ind]))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 114 ms, sys: 55.6 ms, total: 169 ms
Wall time: 2.09 s


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    2.0s finished


In [0]:
gscv.cv_results_

{'mean_fit_time': array([0.04698173, 0.03693652, 0.03917631, 0.03910669, 0.03382055]),
 'mean_score_time': array([0.00165669, 0.00164835, 0.00167902, 0.00165915, 0.00143957]),
 'mean_test_score': array([-0.6880552 , -0.68577622, -0.68010392, -0.66494362, -0.64622507]),
 'param_alpha': masked_array(data=[0.001, 0.01, 0.1, 1, 10],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.001},
  {'alpha': 0.01},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 10}],
 'rank_test_score': array([5, 4, 3, 2, 1], dtype=int32),
 'split0_test_score': array([-0.74052183, -0.73928826, -0.736059  , -0.72651571, -0.71363541]),
 'split1_test_score': array([-0.66967212, -0.6648102 , -0.65658956, -0.63739805, -0.61208545]),
 'split2_test_score': array([-0.65397166, -0.6532302 , -0.6476632 , -0.63091709, -0.61295436]),
 'std_fit_time': array([0.00700515, 0.00209948, 0.00153328, 0.00128192, 0.00547819]),
 'std_score_time': array([6.9

## Now we can fit on all the data and make a prediction

Refit model with best parameters

In [0]:
predictor = Ridge(alpha=1)
predictor.fit(X_train, np.log1p(y_train[training_ind]))

X_test = scaler.transform(df_numeric.iloc[validation_ind])
mean_squared_error(predictor.predict(X_test), np.log1p(y_train[validation_ind]), squared=False)

0.6451176136016002

# Make predictions on the test set



In [0]:
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,0,2013-06-25,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Voskresenskoe,2.149409e+07,7122,0.262459,0.017647,489,NaN,0,469,NaN,0,0,NaN,0,0,0,2,no,0,0,0,no,no,no,no,no,no,no,...,3,1000.00,1500.00,1250.00,0,0,0,3,0,0,0,1,3,1,0,5,0,17.92,4.75,0,0,5,262000,18,700.00,1125.00,912.50,2,4,4,8,0,0,0,1,7,1,0,12,1
1,1,2014-11-20,43,43.0,12.0,20.0,1.0,NaN,1.0,1.0,NaN,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,275,NaN,0,264,NaN,0,0,NaN,0,0,0,0,no,0,1,0,no,no,no,no,no,no,no,...,9,700.00,1222.22,961.11,0,1,5,2,1,0,0,1,4,0,0,6,0,35.62,6.96,1,117300,4,201300,20,747.37,1263.16,1005.26,1,4,8,5,1,1,0,2,12,0,1,11,1
2,2,2014-12-29,73,46.0,9.0,10.0,2.0,1979.0,3.0,10.0,2.0,Investment,Mozhajskoe,1.686153e+07,132349,0.069671,0.130618,9005,4182.0,6,10418,8658.0,6,0,900.0,1,0,12,2,no,0,9,9,no,no,no,yes,no,no,no,...,38,680.56,1138.89,909.72,2,10,12,11,3,0,0,4,4,0,2,21,0,20.20,10.27,19,886563,27,1158392,94,1001.14,1653.41,1327.27,6,17,24,21,15,11,0,8,13,1,2,49,1
3,3,2012-10-03,39,21.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Krjukovo,1.084231e+07,85219,0.062172,0.161532,5767,5278.0,6,5648,10529.0,6,0,30.0,2,0,4,1,no,0,4,1,no,no,no,no,no,no,no,...,6,566.67,916.67,741.67,0,3,1,2,0,0,0,0,2,0,1,4,1,29.70,6.64,3,51038,7,49700,15,540.00,900.00,720.00,0,7,4,4,0,0,0,0,2,0,2,8,1
4,4,2014-11-11,38,19.0,2.0,16.0,5.0,1984.0,1.0,9.0,2.0,Investment,Meshhanskoe,4.512002e+06,57999,0.128162,0.015017,2420,1525.0,3,3079,5752.0,3,0,2890.0,0,1,14,4,no,0,8,59,no,no,no,no,no,no,no,...,1380,864.64,1428.35,1146.49,82,361,335,315,204,71,12,65,127,1,54,73,6,9.72,10.62,725,9341734,111,3278986,2408,876.01,1444.67,1160.34,149,593,601,578,334,127,26,131,235,2,104,209,10


In [0]:
pred_ids = test['id']
test.drop(['id'], axis=1, inplace=True)
X_predict = pd.merge_ordered(test, macro, on='timestamp', how='left')
X_predict.fillna(0, inplace=True)
X_predict = X_predict[df_numeric.columns]

In [0]:
predictions = np.expm1(predictor.predict(scaler.transform(X_predict)))
predictions = pd.DataFrame(predictions, columns=["price_doc"])
predictions = pd.concat([pred_ids, predictions], axis=1)

In [0]:
predictions.to_csv(os.path.join(DATA_PATH, "predictions.csv"), index=False)

In [0]:
!head -n 5 '/content/gdrive/My Drive/mlimperial2020-1/predictions.csv'

id,price_doc
0,5709943.84614118
1,5752432.388775702
2,7190087.855122229
3,7337914.284958483


# Lets use kaggle API again to submit results


In [0]:
!kaggle competitions submit -c mlimperial2020-1 -f "{DATA_PATH}/predictions.csv" -m "Message"

100% 204k/204k [00:04<00:00, 51.3kB/s]
403 - This competition is not currently active.  To test submissions prior to competition launch, use the Sandbox Submissions tool found on the Host tab.
